In [10]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import CountVectorizer
from spacy.tokenizer import Tokenizer
from spacy.vocab import Vocab
from spacy.language import Language

In [3]:
nlp = Language(Vocab())
tokenizer = Tokenizer(nlp.vocab)
def tokenization_process(string):
    aux = tokenizer(string)
    output = list()
    for word in aux:
        output.append(str(word))
    return output

In [4]:
Traintable = pd.read_csv('Recursos/comments_toxicos/Dados/train.csv')

In [5]:
label_cols = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']

In [6]:
TF = CountVectorizer(max_df= 0.7 , min_df= 2, tokenizer = tokenization_process, ngram_range= (1,2),
                     stop_words = 'english')
TermFrequency = TF.fit_transform(Traintable.comment_text)

In [74]:
def CBTW_vec(Term_Frequency_Matrix, Y):
    Y = np.array([Y]).transpose()
    matrix_indexes = Term_Frequency_Matrix.nonzero()
    data_indexes = np.where(Term_Frequency_Matrix.data != 0)[0]
    n_data = len(data_indexes)
    Unitary_matrix = csr_matrix((np.ones(n_data), (matrix_indexes[0][data_indexes], matrix_indexes[1][data_indexes])), shape=Term_Frequency_Matrix.shape)
    A_matrix = Unitary_matrix.multiply(Y)
    A_vector = A_matrix.sum(axis=0).transpose()
    inverse_vec = np.abs(Y-1)
    B_matrix = Unitary_matrix.multiply(inverse_vec)
    B_vector = B_matrix.sum(axis=0).transpose()
    C_vector = np.ones(A_vector.shape[1])*Y.sum()-A_vector
    B_vector[B_vector == 0] = 0.5
    C_vector[C_vector == 0] = 0.5
    return np.squeeze(np.asarray(np.log(1+np.multiply((A_vector/B_vector),(A_vector/C_vector)))))

In [72]:
CBTW_Matrix = np.zeros((TermFrequency.shape[1], len(label_cols)))
for i,j in enumerate(label_cols):
    CBTW_Matrix[:,i] = CBTW_vec(TermFrequency, Traintable[j])
CBTW_Vectors_DF = pd.DataFrame(CBTW_Matrix, columns = label_cols)
CBTW_Vectors_DF.to_csv('CBTW_Vectors.csv', index = False)